In [ ]:
#Start Here for Making Flux Size Graphs
#format beamsize number
import os
os.chdir('/home/peacesea/Burridge2025_WM_AGN/Public/analysis/multi_freq_from_archive')
def format_number(number):
        # Case 1: If the number is less than 1 (e.g., 0.004567)
        if number < 0:
                # Format the positive part of the number and prepend the negative sign
                return "-" + format_number(abs(number))
        if number < 1:
                # Convert the number to a string with high precision
                num_str = f"{number:.16g}"

                # Identify the leading zeros and decimal point
                leading_part = []
                for char in num_str:
                        if char == '0' or char == '.':
                                leading_part.append(char)
                        else:
                                break

                # Remove leading zeros and the decimal point for significant digits
                significant_digits = "".join(char for char in num_str if char.isdigit() and char != "0")

                # Extract the first two non-zero digits
                if len(significant_digits) >= 2:
                        first_two = significant_digits[:2]
                elif len(significant_digits) == 1:
                        first_two = significant_digits + "0"  # Pad with zero if only one significant digit exists
                else:
                        first_two = "00"  # Handle edge case like 0

                # Combine the leading zeros and the first two non-zero digits
                return "".join(leading_part) + first_two

        # Case 2: If the number is greater than or equal to 1
        else:
                # Format to two decimal places
                return f"{number:.2f}"

In [ ]:
#Create Spectral Plots
import matplotlib.pyplot as pylab
import matplotlib.colors
import numpy as np
import shutil
import os
import matplotlib.cm as cmx
import math
from matplotlib.ticker import MultipleLocator
import copy

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

msize=10

path = f'SpectraGraphs'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'
freqs=[]
fluxs=[]
beams=[]

with open(source_text, 'r') as infile:
    jfin=-1
    for line in infile:
        if len(line)<2:
            continue
        jfin=jfin+1 

grouped_data = {}

with open(source_text, 'r') as infile:
    j=-1
    for line in infile:
        if line=='\n':
            continue
        
        line=eval(line)
        source=line[0]
        j=j+1
        
        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            continue


        if source not in grouped_data:
            grouped_data[source] = {
                'sfreqs': [], 'sfluxs': [], 'sbeams': [], 'sdates': [],
                'sfreqs2': [], 'sfluxs2': [], 'sbeams2': [], 'sdates2': [],
            }

        if j==0:
            minfreq=freq
            maxfreq=freq
            minflux=flux
            maxflux=flux
            minbeam=beam
            maxbeam=beam
        if j!=0:
            if freq>maxfreq:
                maxfreq=freq
            if freq<minfreq:
                minfreq=freq
            if flux>maxflux:
                maxflux=flux
            if flux<minflux:
                minflux=flux
            if beam>maxbeam:
                maxbeam=beam
            if beam<minbeam:
                minbeam=beam

minfreqp=round_down_to_half_integer(minfreq)
maxfreqp=round_up_to_half_integer(maxfreq)

minfluxp=round_down_to_half_integer(minflux)
maxfluxp=round_up_to_half_integer(maxflux)

minbeamp=round_down_to_half_integer(minbeam)
maxbeamp=round_up_to_half_integer(maxbeam)

interval = maxfreqp - minfreqp
ten_percent = interval * 0.05
minfreq = minfreqp - ten_percent
maxfreq = maxfreqp + ten_percent

interval = maxfluxp - minfluxp
ten_percent = interval * 0.05
minflux = minfluxp - ten_percent
maxflux = maxfluxp + ten_percent

interval = maxbeamp - minbeamp
ten_percent = interval * 0.05
minbeam = minbeamp - ten_percent
maxbeam = maxbeamp + ten_percent


x_tick_positions=np.arange(minfreqp, maxfreqp + 0.5 , 0.5)  # 6 evenly spaced ticks
x_tick_labels = [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5, 0.5)  # 6 evenly spaced ticks
y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

stick_positions = np.arange(minbeamp, maxbeamp + 1, 1)  # 6 evenly spaced ticks
stick_labels = [f"{pos:.2f}" for pos in stick_positions]  # Convert log to linear

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'

sourcedone=[]
sources=[]

oldsources=[]
oldsource='NA'

with open(source_text, 'r') as infile:
    j=-1
    j2=-1
    for line in infile:
        j2=j2+1 
        if len(line)<2:
            continue
        line=eval(line)

        source=line[0]
        breaker=1
        j=j+1

        if j==0:
            sbeamsizes=[]
            sdates=[]
            sfluxs=[]
            sfreqs=[]
            sbeamsizes2=[]
            sdates2=[]
            sfluxs2=[]
            sfreqs2=[]

            
        breaker=0

        sources.append(source)
        sources=list(set(sources))

        if len(sources)-len(oldsources)!=0 and j!=0:
            source=oldsource

            for isource in sourcedone:
                if source==isource:
                    breaker=1
            if breaker==1:
                continue
            sourcedone.append(source)

            fig, ax = pylab.subplots()
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs/{source}'):
                os.makedirs(f'SpectraGraphs/{source}')
            if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
                os.remove(f'SpectraGraphs/{source}/{nicename}')
            #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
            #pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]

        if j2==jfin-1:
            if line=='\n':
                continue
 
            fig, ax = pylab.subplots()
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs/{source}'):
                os.makedirs(f'SpectraGraphs/{source}')
            if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
                os.remove(f'SpectraGraphs/{source}/{nicename}')
            #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
            #pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


        oldsource=source

        flux_upperb=0
        if len(str(line[3]).split('*'))>1:
            flux_upperb=1

        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        odate=line[4]
        if odate!='NA':
            odate=odate.split('/')
            date=float(odate[0]) + float(odate[1]) / 12 + float(odate[2]) / 365
        else:
            date='NA'
            
        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            continue

        if flux_upperb==0:
            sbeamsizes.append(beam)
            sfluxs.append(flux)
            sfreqs.append(freq)
            sdates.append(date)

        if flux_upperb==1:
            sbeamsizes2.append(beam)
            sfluxs2.append(flux)
            sfreqs2.append(freq)
            sdates2.append(date)

        oldsources=copy.deepcopy(sources)


In [ ]:
# functino to Group frequencies dynamically
def group_by_frequency(freqs, fluxes, beams, dates, tolerance):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    current_group_freqs, current_group_fluxes, current_group_beams, current_group_dates = [], [], [], []

    sorted_data = sorted(zip(freqs, fluxes, beams, dates), key=lambda x: x[0])
    for freq, flux, beam, date in sorted_data:
        if not current_group_freqs or abs(freq - current_group_freqs[0]) <= tolerance:
            current_group_freqs.append(freq)
            current_group_fluxes.append(flux)
            current_group_beams.append(beam)
            current_group_dates.append(date)
        else:
            grouped_freqs.append(current_group_freqs)
            grouped_fluxes.append(current_group_fluxes)
            grouped_beams.append(current_group_beams)
            grouped_dates.append(current_group_dates)
            current_group_freqs = [freq]
            current_group_fluxes = [flux]
            current_group_beams = [beam]
            current_group_dates = [date]

    if current_group_freqs:
        grouped_freqs.append(current_group_freqs)
        grouped_fluxes.append(current_group_fluxes)
        grouped_beams.append(current_group_beams)
        grouped_dates.append(current_group_dates)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Define frequency ranges for VLA and ALMA bands
vla_bands = [
    (0.0,0.054) ,     #for Rounding 
    (0.054, 0.086),  # 4 Band
    (0.230, 0.470),  # P Band
    (1, 2),          # L Band
    (2, 4),          # S Band
    (4, 8),          # C Band
    (8, 12),         # X Band
    (12, 18),        # Ku Band
    (18, 26.5),      # K Band
    (26.5, 40),      # Ka Band
    (40, 50),        # Q Band
]

alma_bands = [
    (31.3, 45),   # Band 1
    (67, 90),     # Band 2
    (84, 116),    # Band 3
    (125, 163),   # Band 4
    (163, 211),   # Band 5
    (211, 275),   # Band 6
    (275, 373),   # Band 7
    (385, 500),   # Band 8
    (602, 720),   # Band 9
    (787, 950),   # Band 10
]

# Combine all boundaries and create continuous intervals
all_boundaries = sorted(set([freq for band in vla_bands + alma_bands for freq in band]))
non_overlapping_intervals = [(all_boundaries[i], all_boundaries[i + 1]) for i in range(len(all_boundaries) - 1)]

# Remove gaps by ensuring no holes
continuous_intervals = []
for i, (start, end) in enumerate(non_overlapping_intervals):
    if i > 0 and start > continuous_intervals[-1][1]:
        # Fill the gap between the previous interval and the current one
        continuous_intervals.append((continuous_intervals[-1][1], start))
    continuous_intervals.append((start, end))

# Function to check if a frequency is in any interval
def is_frequency_in_intervals(frequency, intervals):
    for start, end in intervals:
        if start <= frequency < end:  # Check if frequency is in the interval
            return True, (start, end)
    return False, None

# Print all intervals for verification
print("\nAll continuous intervals:")
for i, interval in enumerate(continuous_intervals):
    print(f"Interval {i + 1}: {interval[0]}–{interval[1]} GHz")

def group_by_band(freqs, fluxes, beams, dates, intervals):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    interval_mapping = {interval: ([], [], [], []) for interval in intervals}

    for freq, flux, beam, date in zip(freqs, fluxes, beams, dates):
        found, assigned_interval = is_frequency_in_intervals(freq, intervals)
        if found:
            interval_mapping[assigned_interval][0].append(freq)
            interval_mapping[assigned_interval][1].append(flux)
            interval_mapping[assigned_interval][2].append(beam)
            interval_mapping[assigned_interval][3].append(date)

    for interval, (freq_list, flux_list, beam_list, date_list) in interval_mapping.items():
        if freq_list:
            grouped_freqs.append(freq_list)
            grouped_fluxes.append(flux_list)
            grouped_beams.append(beam_list)
            grouped_dates.append(date_list)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Generate Statistical Spectral Plots
# Define tolerance for grouping frequencies (log space)
tolerance = .05

#define if it should be grouped dynamicall or by band
grouper='byband'
#grouper='bydynam'

# Generate consistent x-axis and y-axis tick positions
x_tick_positions = np.linspace(minfreq, maxfreq, num=6)  # 6 evenly spaced ticks
x_tick_labels = [f"{10**pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.linspace(minflux, maxflux, num=6)  # 6 evenly spaced ticks
y_tick_labels = [f"{10**pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

# Apply grouping to all sources
for source, data in grouped_data.items():
    if grouper=='bydynam':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_frequency(
            data['sfreqs'], data['sfluxs'], data['sbeams'],data['sdates'],tolerance
        )
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_frequency(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'],tolerance
        )

    if grouper=='byband':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_band(
            data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'], continuous_intervals
        )
        
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_band(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'], continuous_intervals
        )


# Initialize combined arrays
combined_freqs = []  # Combined frequencies
combined_fluxes = []  # Combined fluxes
combined_beams = []  # Combined beam sizes

for source, data in grouped_data.items():
    # Combine all frequency, flux, and beam data for the current source
    combined_freqs = data['sfreqs'] 
    combined_fluxes = data['sfluxs'] 
    combined_beams = data['sbeams'] 
    combined_dates = data['sdates'] 

    combined_freqs_2 = data['sfreqs2']
    combined_fluxes_2 = data['sfluxs2'] 
    combined_beams_2 = data['sbeams2'] 
    combined_dates_2 = data['sdates2'] 

    # Group the combined data dynamically for the current source
    if grouper=='bydynam':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_frequency(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, tolerance
        )

    if grouper=='byband':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_band(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, continuous_intervals
        )

    # Iterate over grouped combined data
    for freq_group, flux_group, beam_group in zip(grouped_freqs, grouped_fluxes, grouped_beams):
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        if group_size >= 4:
            pylab.boxplot(flux_group, positions=[central_freq],widths=0.1,
            showfliers=False)  # Customize outliers
        elif group_size == 3:
            # Vertical line with caps at min and max
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = np.median(flux_group)
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5,  # Length of caps
                color='k'
            )
            # Add a horizontal line at the median_flux
            pylab.hlines(
                y=median_flux,  # Position of the horizontal line
                xmin=central_freq - 0.05,  # Adjust as needed for length
                xmax=central_freq + 0.05, 
                color='r',  # Color of the line
                linewidth=1  # Thickness of the horizontal line
            )
        elif group_size == 2:
            # Vertical line with caps at min and max (no median)
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = (min_flux + max_flux) / 2  # Just for centering the line
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5, 
                color='k'
            )

        elif group_size == 1:
            pylab.scatter([central_freq], [flux_group[0]], c='b', marker='o', s=10)

    if combined_fluxes_2:  # Ensure '2' fluxes exist
        if not combined_fluxes or min(combined_fluxes_2) < min(combined_fluxes):
            # If main fluxes are empty OR '2' minimum is lower
            min_flux_2 = min(combined_fluxes_2)
            min_flux_2_index = combined_fluxes_2.index(min_flux_2)
            min_freq_2 = combined_freqs_2[min_flux_2_index]
            # Mark the lowest value with a downward triangle
            pylab.scatter([min_freq_2], [min_flux_2], c='black', marker='v', s=7, label='Min Flux from 2')


    # Set consistent x and y limits for all plots
    pylab.xlim(minfreq - 0.1, maxfreq + 0.1)  # Add slight padding
    pylab.ylim(minflux - 0.1, maxflux + 0.1)  # Add slight padding
    # Set consistent x-axis ticks and labels
    # Set consistent x-axis and y-axis ticks and labels
    pylab.xticks(x_tick_positions, x_tick_labels)
    pylab.yticks(y_tick_positions, y_tick_labels)

    # Add labels and title
    pylab.xlabel('Frequency [GHz]')
    pylab.ylabel(r'Flux [mJy][Beam]$^{-1}$')
    pylab.title(f'{source}')

    nicename=f'{source} statistical spectra.pdf'
    if not os.path.exists(f'SpectraGraphs/{source}'):
        os.makedirs(f'SpectraGraphs/{source}')
    if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
        os.remove(f'SpectraGraphs/{source}/{nicename}')
    #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
    #pylab.show()
    pylab.clf() 

In [ ]:
#NEW make function to get distances
def process_sourcenew(source_name):
    found=0
    for idist in totdist:
        if idist[0].lower() in source_name.lower():
            dist=idist[1]
            dist_uncer=idist[2]
            found=found+1
    if found!=1:
        print(found)
        input(f'error with finding distance for {source_name}')
    #return in MPc
    return([dist,dist_uncer])

#distances for Pesce, 2018 ordered to Burridge, 2025
import numpy as np

totdist=[]

hubble=73.9
hubbleerr=3.0

val=466.87
valerr=0.49
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC4258",dist,disterr])
print(f"NGC 4258: {dist:.2f} ± {disterr:.2f}")

val=4088.8
valerr=5.3
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC1194",dist,disterr])
print(f"NGC1194: {dist:.1f} ± {disterr:.1f}")

val=8322.22
valerr=1.13
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC5765b",dist,disterr])
print(f"NGC 5765b: {dist:.0f} ± {disterr:.0f}")

val=1850.8
valerr1=13.5
valerr2=13.9
dist=val/hubble
disterr1=np.sqrt((valerr1/hubble)**2+(hubbleerr*val/hubble**2)**2)
disterr2=np.sqrt((valerr2/hubble)**2+(hubbleerr*val/hubble**2)**2)
disterr=(disterr1+disterr2)/2
totdist.append(["NGC2273",dist,disterr])
print(f"NGC2273: {dist:.1f} ± {disterr:.1f}")

val=10189.26
valerr=1.20
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC6264",dist,disterr])
print(f"NGC6264: {dist:.0f} ± {disterr:.0f}")

val=3259.75
valerr=1.00
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["UGC3789",dist,disterr])
print(f"UGC3789: {dist:.1f} ± {disterr:.1f}")

val=4954.5
valerr=15
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC2960",dist,disterr])
print(f"NGC2960: {dist:.1f} ± {disterr:.1f}")

val=7618.2
valerr=14.0
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["ESO558",dist,disterr])
print(f"ESO558: {dist:.0f} ± {disterr:.0f}")

val=7834.28
valerr1=2.1
valerr2=2.2
dist=val/hubble
disterr1=np.sqrt((valerr1/hubble)**2+(hubbleerr*val/hubble**2)**2)
disterr2=np.sqrt((valerr2/hubble)**2+(hubbleerr*val/hubble**2)**2)
disterr=(disterr1+disterr2)/2
totdist.append(["NGC6323",dist,disterr])
print(f"NGC6323: {dist:.0f} ± {disterr:.0f}")

val=4818
valerr=10.5
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["J0437",dist,disterr])
print(f"J0437: {dist:.1f} ± {disterr:.1f}")

#distance for other sources 

#NGC 1068 
#Baer-Way, 2024
#2024ApJ...964..172B
val=1137
valerr=3
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC1068",dist,disterr])
print(f"NGC1068: {dist:.1f} ± {disterr:.1f}")

#NGC 3393 
#Koss, 2022
#2022ApJS..261....6K
val=3833
valerr=5
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC3393",dist,disterr])
print(f"NGC3393: {dist:.1f} ± {disterr:.1f}")

#Circinus
#Tully, 2022
#2009AJ....138..323T
mM=28.12
mMerr=0.36
dist=10**((mM+5)/5)/10**6
disterr=np.log(10)/5*dist*mMerr
totdist.append(["Circinus",dist,disterr])
print(f"Circinus: {dist:.3f} ± {disterr:.3f}")

#CGCG 074-06
#Albareti, 2017
#2016SDSSD.C...0000:
val=6915
valerr=3
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["CGCG074-06",dist,disterr])
print(f"CGCG074-06: {dist:.1f} ± {disterr:.1f}")

#NGC 4388
#Lu, 1993
#1993ApJS...88..383L
val=2524
valerr=1
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC4388",dist,disterr])
print(f"NGC4388: {dist:.1f} ± {disterr:.1f}")

#NGC 4945
#Kanehisa, 2023
#2023MNRAS.519.6184K
val=563
valerr=3
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC4945",dist,disterr])
print(f"NGC4945: {dist:.1f} ± {disterr:.1f}")

#UGC 6093
#Albareti, 2017
#2016SDSSD.C...0000:
val=10803
valerr=3
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["UGC6093",dist,disterr])
print(f"UGC6093: {dist:.0f} ± {disterr:.0f}")

#NGC 3079
#Springbob, 2005
#2005ApJS..160..149S
val=1106
valerr=1
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC3079",dist,disterr])
print(f"NGC3079: {dist:.1f} ± {disterr:.1f}")

#NGC 1320
#Theureau, 2017
#2017A&A...599A.104T
val=2783
valerr=10
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["NGC1320",dist,disterr])
print(f"NGC1320: {dist:.1f} ± {disterr:.1f}")

#IC 2560
#Pesce, 2015
#2015ApJ...810...65P
val=2925
valerr=2
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["IC2560",dist,disterr])
print(f"IC2560: {dist:.1f} ± {disterr:.1f}")

#Mrk 1029
#Huchra, 1999
#1999ApJS..121..287H
val=9076
valerr=32
dist=val/hubble
disterr=np.sqrt((valerr/hubble)**2+(hubbleerr*val/hubble**2)**2)
totdist.append(["Mrk1029",dist,disterr])
print(f"Mrk1029: {dist:.0f} ± {disterr:.0f}")

In [ ]:
#OLD make function to get distances

import os
from astroquery.ned import Ned
import ast  # For safely parsing text file data

import os
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

from astropy.coordinates import SkyCoord
import numpy as np
import astropy.units as u

import re

import requests
import csv
from scipy import constants
c=constants.c

def is_subsequence(small, large):
    it = iter(large)
    return all(char in it for char in small)

def decompose_string(s):
    parts = re.findall(r"[A-Za-z]+|\d+", s)  # Finds all letter and number sequences
    return parts if parts else [s]

# Input file with source names and coordinates
txtnamesandcoords = 'namesandcoords.txt'

# List of desired reference codes
#2018ApJ...863.. (Pesce2018) is the only one found
wantrefs = ['2018ApJ...863..', '2023ApJ...948..', '2020ApJ...890..', '2020ApJ...891..']

def getdistsold(newnames):
    newrefs=[]
    if not os.path.exists('distancetable.csv'):
        url = 'https://ned.ipac.caltech.edu/Archive/Distances/NED30.5.1-D-17.1.2-20200415.csv'
        response = requests.get(url)

        if response.status_code == 200:
            with open('distancetable.csv', "wb") as file:
                file.write(response.content)
        else:
            return(f"Failed to download. Status code: {response.status_code}")
    
    maserdist=[]
    otherdist=[]
    j1=-1
    j2=-1
    with open('distancetable.csv', "r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            textsource=row[3]
            if len(textsource)==0:
                continue
            textdistmm=row[4]
            textdistmmerr=row[5]

            textdistMpc=row[6]
            textmethod=row[7]
            refcode=row[8]


            textsource=textsource.replace(' ','')

            findred=0
            for i in newnames:
                checkers=[]
                for ii in i:
                    ii=ii.split('Galaxy')[0]
                    if is_subsequence(ii, textsource):
                        checkers.append(ii)
                if len(checkers)==len(i):
                    if row[10]:
                        findred=1
                        redshift=row[10]
                        Hubble=row[11]
                        newdist=c*float(redshift)/float(Hubble)*c
                    
                    if textdistmmerr:
                        if float(textdistmmerr)>0:
                            if 'maser' in textmethod.lower():
                                j1=j1+1
                                newrefs.append(refcode)
                                if j1==0:
                                        maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                        maserdisuncer=textdistmmerr
                                elif textdistmmerr<maserdisuncer:
                                    maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                            else:
                                j2=j2+1
                                if j2==0:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                    otherdisuncer=textdistmmerr
                                elif textdistmmerr<otherdisuncer:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]

    return(maserdist,otherdist,refcode,findred,newrefs)

# Function to process each source
def process_sourceold(source_name, wantrefs, ra, dec,threshold, secondarynames):
  
    """Query NED for redshift data and filter based on references and uncertainties."""
    # Skip excluded sources

    # Query the redshift table from NED
    distances_table = Ned.get_table(source_name, table='redshifts')

    # Extract relevant columns
    redshifts = distances_table['Published Redshift']
    uncertainties = distances_table['Published Redshift Uncertainty']
    refcodes = distances_table['Refcode']

    # Combine into tuples
    entries = [
        (redshift, uncertainty, refcode)
        for redshift, uncertainty, refcode in zip(redshifts, uncertainties, refcodes)
    ]

    # Filter for entries with desired references
    desired_entries = [
        entry for entry in entries if any(ref in entry[2] for ref in wantrefs)
    ]

    entries_with_reference = []
    entries_without_reference = []

    for redshift, uncertainty, refcode in desired_entries:
        if any(ref in refcode for ref in wantrefs):  # Check if refcode contains a wanted reference
            entries_with_reference.append((redshift, uncertainty, refcode))
        else:
            entries_without_reference.append((redshift, uncertainty, refcode))


    reduced_entries = {}

    # Step 2a: Prioritize entries from `entries_with_reference`
    if entries_with_reference:
        for redshift, uncertainty, refcode in entries_with_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)

    if len(reduced_entries)==0:
        # Step 2b: If `entries_with_reference` is empty, use `entries_without_reference`
        for redshift, uncertainty, refcode in entries_without_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)



    # Print results based on the filtering
    redshift='NA'
    uncertainty='NA'

    newnames=[]
    newlist=decompose_string(secondarynames[0][0])

    newnames.append(newlist)
    if len(secondarynames[0][1])!=0:
        if secondarynames[0][1][0]!='NA':
            for i in secondarynames[0][1]:
                if not is_subsequence(secondarynames[0][0], i):
                    newlist=decompose_string(i)
                    newnames.append(newlist)

    maserdists, otherdists, inewrefs, findred, newrefs = getdistsold(newnames)

    if reduced_entries:
        for irefcode, (iredshift, iuncertainty) in reduced_entries.items():
            redshift=iredshift
            ynfindref='y'
            refcode=irefcode
            if iuncertainty>0:
                uncertainty=iuncertainty
            else:
                input(f'No uncertainty for chosen referencecode: {refcode} , {source}')
                uncertainty='NA'
    else:

        # If no desired references, and no maser distances, find valid entries with uncertainties
        valid_entries = [entry for entry in entries if entry[1] > 0]
        if valid_entries:
            # Select the entry with the lowest uncertainty
            lowest_uncertainty_entry = min(valid_entries, key=lambda x: x[1])
            lowest_redshift, lowest_uncertainty, reference_code = lowest_uncertainty_entry
            redshift=lowest_redshift
            uncertainty= lowest_uncertainty
            refcode=reference_code
            ynfindref='n'
        else:
            input(f"No valid redshift entries found for {source_name} (1)\n")
        if len(maserdists)!=0:
            z_observed = redshift
            if z_observed >= threshold:
                if len(maserdists)!=0 and findred==1:
                        input('new maser source found... adjust accordingly (1)')
        
    z_observed = redshift
    z_uncer=uncertainty

    if z_observed <= threshold:
        if len(maserdists)!=0:
            input('new maser source found... adjust accordingly (2)')
        if len(maserdists)==0:
            textdistmm=float(otherdists[1])
            textdistmmerr=float(otherdists[2])
            textdistMpc=float(otherdists[3])

            newdist=10**((textdistmm)/5-5)
            newdist_uncer=np.log(10)/5*newdist*textdistmmerr
            textmethod=otherdists[4]
            refcode=otherdists[5]

            return([newdist,newdist_uncer],textmethod,'NA',refcode,'DISTANCE',newrefs)

    # Calculate heliocentric velocity
    c = 299792.458  # Speed of light in km/s
    v_helio = z_observed * c
    v_helio_uncer = z_uncer * c

    #Hubble Constant
    #Pesce, APJ 891:L1
    H = 73.9
    H_uncer = 3
    dist=v_helio/H
    dist_uncer=np.sqrt((v_helio_uncer/H)**2 + (v_helio/H**2*H_uncer)**2)

    return([dist,dist_uncer],'NA',ynfindref,refcode,'REDSHIFT', newrefs)

In [ ]:
#Path on local computer to SMBH mass
from pathlib import Path
parent_dir = Path.cwd().parent
Greenepath=f'{parent_dir}/CountingBHS_and_Spectra/Greene_params/Greene_params.txt'

In [ ]:
#date format
from datetime import datetime, timedelta

def decimal_year_to_date(decimal_year):
    year = int(decimal_year)  # Extract the year
    fraction = decimal_year - year  # Extract the decimal part

    # Convert the decimal part to days (assuming 365 days in a year)
    days_elapsed = int(round(fraction * 365))  

    # Compute the actual date by adding days to January 1st of that year
    date = datetime(year, 1, 1) + timedelta(days=days_elapsed)

    # Format as YY.MM.DD
    return f"{date.year }/{date.month:02}/{date.day:02}"

In [ ]:
#get SMBH size
from scipy import constants
from astropy import constants as aconstants
from matplotlib.lines import Line2D

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
G=constants.gravitational_constant
pi=constants.pi

def linear_to_log_uncertainty(x, delta_x):
    return delta_x / (x * np.log(10))

def log_to_linear_uncertainty(x, delta_x):
    return x * np.log(10) * delta_x

def getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc):
    mass=10**float(log_BH_mass_solar)*mass_sun
    mass_unc=mass*np.log(10)*float(log_BH_mass_solar_uncer)
    mass_fracerror=(mass_unc/mass)*100

    distance=distanceMPc*pc*10**6
    distance_uncer=distanceMPc_uncer*pc*10**6
    dist_fracerror=(distance_uncer/distance)*100
    
    if arcsectopc=='NA':
        m_d=mass/distance
        m_d_unc=np.sqrt((m_d/mass*mass_unc)**2+(m_d/distance*distance_uncer)**2)
        m_d_fracerror=(m_d_unc/m_d)*100
        diam_bhs_microarcsec_fracerror = m_d_fracerror

        diam_bhs_rad=2*np.sqrt(27)*G/(c**2)*m_d
        diam_bhs_deg=diam_bhs_rad*360/(2*pi)
        diam_bhs_arcsec=diam_bhs_deg*60*60
        diam_bhs_microarcsec=diam_bhs_arcsec*10**6

        diam_bhs_microarcsec_unc=diam_bhs_microarcsec/(m_d)*m_d_unc


        return(log_BH_mass_solar, log_BH_mass_solar_uncer, mass_fracerror, distanceMPc, distanceMPc_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

        
    diam_bhs=2*np.sqrt(27)*G/(c**2)*mass
    diam_bhs_unc = 2*np.sqrt(27)*G/(c**2)*mass_unc
    
    diam_bhspc=diam_bhs/pc
    diam_bhspc_unc=diam_bhs_unc/pc

    diam_bhs_arsec=diam_bhspc/arcsectopc
    diam_bhs_microarcsec=diam_bhs_arsec*10**6

    diam_bhs_arsec_unc=np.sqrt(
                                (diam_bhspc_unc/arcsectopc)**2
                                + ((-1)*diam_bhspc/arcsectopc**2*arcsectopc_unc)**2
                                )
    
    diam_bhs_microarcsec_unc=diam_bhs_arsec_unc*10**6
    diam_bhs_microarcsec_fracerror = (diam_bhs_microarcsec_unc/diam_bhs_microarcsec)*100

    return(mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)


sizeIC1459=8.9
sizeNGC4594=7.0
logIC1459flux=np.log10(220)
logNGC4594flux=np.log10(200)

logIC1459fluxunc=linear_to_log_uncertainty(220, 21)
logNGC4594fluxunc=linear_to_log_uncertainty(200, 10)

#IC1459
distanceMPc=29
distanceMPc_uncer=3.7

log_BH_mass_solar=9.4
log_BH_mass_solar_uncer_up=0.077
log_BH_mass_solar_uncer_down=0.035

mass, mass_unc_up, mass_fracerror_up, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer_up,distanceMPc,distanceMPc_uncer,'NA','NA')
mass, mass_unc_down, mass_fracerror_down, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer_down,distanceMPc,distanceMPc_uncer,'NA','NA')

logIC1459sizeunc_up=linear_to_log_uncertainty(diam_bhs_microarcsec, diam_bhs_microarcsec_uncup)
logIC1459sizeunc_down=linear_to_log_uncertainty(diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown)
logIC1459sizeunc=np.array([[logIC1459sizeunc_down], [logIC1459sizeunc_up]])
sizeIC1459=diam_bhs_microarcsec

#NGC4594
distanceMPc=9.9
distanceMPc_uncer=0.82

log_BH_mass_solar=8.8
log_BH_mass_solar_uncer_up=0.025
log_BH_mass_solar_uncer_down=0.027

mass, mass_unc_up, mass_fracerror_up, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer_up,distanceMPc,distanceMPc_uncer,'NA','NA')
mass, mass_unc_down, mass_fracerror_down, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer_down,distanceMPc,distanceMPc_uncer,'NA','NA')

logNGC4594sizeunc_up=linear_to_log_uncertainty(diam_bhs_microarcsec, diam_bhs_microarcsec_uncup)
logNGC4594sizeunc_down=linear_to_log_uncertainty(diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown)
logNGC4594sizeunc=np.array([[logNGC4594sizeunc_down], [logNGC4594sizeunc_up]])
sizeNGC4594=diam_bhs_microarcsec



m87size=diam_bhs_microarcsec
m87sizeunc_down=diam_bhs_microarcsec_uncdown
m87sizeunc_up=diam_bhs_microarcsec_uncup
logm87sizeunc_down=linear_to_log_uncertainty(m87size, m87sizeunc_down)
logm87sizeunc_up=linear_to_log_uncertainty(m87size, m87sizeunc_up)

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)






sagmass=6.6
sagmass_unc_up=.12
sagmass_unc_down=.07
sagdist=8.15*10**(-3)
sagdist_unc=.15*10**(-3)

distanceMPc=sagdist
distanceMPc_uncer=sagdist_unc

log_BH_mass_solar=sagmass
log_BH_mass_solar_uncerup=sagmass_unc_up
log_BH_mass_solar_uncerdown=sagmass_unc_down

mass, mass_uncup, mass_fracerrorup, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(sagmass,sagmass_unc_up,sagdist,sagdist_unc,'NA','NA')
mass, mass_uncdown, mass_fracerrordown, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(sagmass,sagmass_unc_down,sagdist,sagdist_unc,'NA','NA')

sagsize=diam_bhs_microarcsec
sagsizeunc_down=diam_bhs_microarcsec_uncdown
sagsizeunc_up=diam_bhs_microarcsec_uncup
logsagsizeunc_down=linear_to_log_uncertainty(sagsize, sagsizeunc_down)
logsagsizeunc_up=linear_to_log_uncertainty(sagsize, sagsizeunc_up)

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'SagA*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} ± {format_number(distanceMPc_uncer)} MPc ({format_number(dist_fracerror)}%)', linestyle='')
log_BH_mass_solar_uncerup=format_number(float(log_BH_mass_solar_uncerup))
log_BH_mass_solar_uncerdown=format_number(float(log_BH_mass_solar_uncerdown))
mass_fracerror=(mass_fracerrorup+mass_fracerrordown)/2
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} $^{{+{log_BH_mass_solar_uncerup}}}_{{-{log_BH_mass_solar_uncerdown}}}$ log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles1=[sourcehan,sizechan,masshan,disthan]

fig, ax = pylab.subplots()

m87mass=9.81
m87mass_unc=0.06
m87dist=16.8
m87dist_uncup=.8
m87dist_uncdown=.7

distanceMPc=m87dist
distanceMPc_uncerup=m87dist_uncup
distanceMPc_uncerdown=m87dist_uncdown

log_BH_mass_solar=m87mass
log_BH_mass_solar_uncer=m87mass_unc

mass, mass_unc, mass_fracerror, distance, distance_uncerup, dist_fracerrorup, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncup,'NA','NA')
mass, mass_unc, mass_fracerror, distance, distance_uncerdown, dist_fracerrordown, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncdown,'NA','NA')

m87size=diam_bhs_microarcsec
m87sizeunc_down=diam_bhs_microarcsec_uncdown
m87sizeunc_up=diam_bhs_microarcsec_uncup
logm87sizeunc_down=linear_to_log_uncertainty(m87size, m87sizeunc_down)
logm87sizeunc_up=linear_to_log_uncertainty(m87size, m87sizeunc_up)

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'M87*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
dist_fracerror=(distanceMPc_uncerup+distanceMPc_uncerdown)/2
distanceMPc_uncerup=format_number(float(distanceMPc_uncerup))
distanceMPc_uncerdown=format_number(float(distanceMPc_uncerdown))
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} $^{{+{distanceMPc_uncerup}}}_{{-{distanceMPc_uncerdown}}}$ MPc ({format_number(dist_fracerror)}%)', linestyle='')
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} ±{log_BH_mass_solar_uncerdown} log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles2=[sourcehan,sizechan,masshan,disthan]

legend1=ax.legend(handles=handles2, loc='upper center')   
ax.add_artist(legend1)
ax.legend(handles=handles1, loc='lower center')   
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks

# Remove tick labels
ax.set_xticklabels([])  # Remove x-axis labels
ax.set_yticklabels([])  # Remove y-axis labels
if not os.path.exists('Extras'):
    os.makedirs('Extras')
#pylab.savefig(f'Extras/SMBH of SagA* and M87')
pylab.clf() 


In [ ]:
#Gather data for BHS vs Flux Graph
import matplotlib.pyplot as plt

findwithprocessor=0

noprocessor=[
    ['4258',0.617,0.049],
    ['1194',0.131,0.016],
    ['1068',0.111,0.0005],
    ['5765b',0.0398,0.0049],
    ['2273',0.0349,0.0035],
    ['3393',0.0314,0.0239],
    ['circinus',0.0280,0.0079],
    ['074-064',0.0265,0.0026],
    ['6264',0.0230,0.0028],
    ['3789',0.0227,0.0048],
    ['4388',0.0218,0.0022],
    ['4945',0.0186,0.0077],
    ['6093',0.0180,0.0011],
    ['3079',0.0172,0.0021],
    ['558',0.0165,0.0013],
    ['2960',0.0164,0.0020],
    ['0437',0.0157,0.0023],
    ['1320',0.0150,0.0056],
    ['2560',0.0113,0.0016],
    ['6323',0.00968,0.00118],
    ['1029',0.00159,0.00048]
    ]

chooser=False
choosesource='1029'

threshold=0.0015

def find_flux_with_smallest_beam(arrays):
    # Find the index of the smallest beam size in the array
    arrays = np.array(arrays)
    index_of_smallest_beam = np.argmin(arrays[:, 2].astype(float))
    flux = arrays[index_of_smallest_beam, 0]
    fluxunc = arrays[index_of_smallest_beam, 1]
    return flux, fluxunc

def find_sma(arrays):
    # Find the index of the smallest beam size in the array
    arrays = np.array(arrays)
    index_of_smallest_beam = np.argmin(arrays[:, 2].astype(float))
    flux = arrays[index_of_smallest_beam, 0]
    fluxunc = arrays[index_of_smallest_beam, 1]
    return flux, fluxunc

def linear_to_log_uncertainty(x, delta_x):
    return delta_x / (x * np.log(10))

graph_array=[]
nograph=[]

for isource, data in grouped_data.items():
    if isource=='NGC5495':
        continue
    if chooser==True and choosesource not in isource:
        continue
    # Main loop to read and process sources from the input file
    if os.path.exists(txtnamesandcoords):
        with open(txtnamesandcoords, 'r') as infile:
            for line in infile:

                # Parse the line into a list of name-coordinate pairs
                params = ast.literal_eval(line)
                for inamecoords in params:                
                    source_name = inamecoords[0][0]
                    if not isource.lower() in source_name.lower():
                        continue
                    source = source_name
                    ra = inamecoords[1][1]
                    dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                    distanceMPc, distanceMPc_uncer = process_sourcenew(source_name)

                    method='NA'
                    ynfindref='NA'
                    refcode='NA'
                    red_depend='NA'

                    #OLD METHOD 
                    #findvals=process_sourceold(source_name, wantrefs, ra, dec, threshold, inamecoords)
                    #method=findvals[1]
                    #ynfindref=findvals[2]
                    #refcode=findvals[3]
                    #red_depend=findvals[4]

                    #print(isource)
                    #print(findvals[3])
                    #print(f"{distanceMPc:.1f}")
    else:
        print(f"File {txtnamesandcoords} does not exist.")

    arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
    arcsectopc=arcsectokpc*1000

    arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
    arcsectopc_unc=arcsectokpc_error*1000

    arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

    distance='NA'
    with open(Greenepath, 'r') as infile:
        j=-1
        breaker=1
        for line in infile:
            j=j+1
            if j>20:
                continue
            source_name=line.split(' ')[0]
            source_name=source_name.replace("−","-")
            if isource=='WISEAJ043703':
                isource='J0437'
            if isource.lower() in source_name.lower():
                log_BH_mass_solar=line.split(' ')[3]
                log_BH_mass_solar_offset=line.split(' ')[5]
                breaker=0
            if 'cgcg' in isource.lower():
                log_BH_mass_solar=7.38
                log_BH_mass_solar_offset=0.012
                breaker=0
        if breaker!=1:
            if findwithprocessor==1:
                mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)
            else:
                finder=0
                for findsource,bhs,bhsunc in noprocessor:
                    if findsource.lower() in isource.lower():
                        diam_bhs_microarcsec=bhs
                        diam_bhs_microarcsec_unc=bhsunc
                        finder=1
                if finder==0:
                    input(f"ERROR for {isource} not found in noprocessor list")
        else:
            print(f'{source} not found')
        

        fluxs=[]
        fluxsuplim=[]
        teles=[]
        with open(source_text, 'r') as infile:
            j=-1
            for line in infile:
                if line=='\n':
                    continue
                line=eval(line)
                source=line[0]
                if isource in source:
                    snr=line[6].split('\n')[0]

                    freq=np.log10(float(line[2]))
                    flux=np.log10(float(str(line[3]).split('*')[0]))
                    if snr!='NA':
                        linflux=float(line[3].split('*')[0])
                        linfluxunc=float(linflux)/float(snr)
                        #def linear_to_log_uncertainty(x, delta_x):

                        fluxunc=linear_to_log_uncertainty(linflux,linfluxunc)
                    else:
                        fluxunc='NA'
                        
                    if '-' not in line[5]:
                        beam=np.log10(float(line[5]))
                    else:
                        continue

                    lim=[200,400]

                    if 10**freq<lim[1] and 10**freq>lim[0]:
                        fluxs.append([flux,fluxunc,beam])

                    tele=line[1]

                    if tele=='SMA':
                        teles.append([flux,fluxunc])
        if fluxs:      
            graph_array.append(isource)
            flux_with_smallest_beam = find_flux_with_smallest_beam(fluxs)
            sma_flux = find_sma(fluxs)
            graph_array.append(flux_with_smallest_beam)
            if diam_bhs_microarcsec_unc!='NA':
                log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
            else:
                log_diam_bhs_microarcsec_unc = 'NA'
            log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
            bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
            graph_array.append(bhs)
            smasize=np.size(teles)
            if smasize==4 and '4258'in isource:
              graph_array.append(teles)
              print('test')
            elif smasize==2:
              graph_array.append(teles)
              pass
            elif smasize==0:
              graph_array.append(['NA','NA'])
              pass
            else:
                print(source)
                print(smasize)
        else:
            log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
            log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
            bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
            nograph.append([isource,bhs])
            smasize=np.size(teles)
            if smasize==2:
              input('error')
            elif smasize==0:
              nograph.append(['NA','NA'])
              pass
            else:
                input('error2')

In [ ]:
#Make BHS vs Flux Graph

sourcesofinterest=('4258','1194','3079','4945','Circinus','1068')
allsourcesofinterest=False

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

earthdiamval=12756
earthradval=earthdiamval/2


#in km
#https://www.nasa.gov/wp-content/uploads/2009/07/180561main_ETM.Distance.Moon_.pdf
#average moon
moondist = ['Moon',382500+earthradval]
#at the equator
#https://imagine.gsfc.nasa.gov/features/cosmic/earth_info.html
earthdiam = ['Earth',earthdiamval]
#https://science.nasa.gov/mission/webb/orbit/#:~:text=A%20Solar%20Orbit&text=Webb%20orbits%20the%20sun%201.5,second%20Lagrange%20point%20or%20L2.
l2dist = ['L2',1.5*10**6+earthradval]
#https://science.nasa.gov/mars/facts/
marsdist = ['Mars',228*10**6+earthradval]
bhex = ['BHEX', 20200+earthradval]

sun_dist_l3  = ['L3', 300*10**6+earthradval]
l45  = ['L4/L5', 150*10**6+earthradval]

#in GHz
operfreq=230

baselines=[moondist,earthdiam,l2dist,bhex,l45]

baselinesize=[]
for ibase in baselines:
    operfreq_Hz=operfreq*10**9
    wavelength=c/operfreq_Hz
    bl_size=wavelength/(ibase[1]*1000)
    bl_size=bl_size*180/pi*3600*10**6
    baselinesize.append([ibase[0],bl_size])



fluxthresh=[0.1,1,10,100]


# Organizing data into a dictionary
data=graph_array


galaxy_data = {}
for i in range(0, len(data), 4):
    galaxy_name = data[i]
    flux_beam = data[i+1]
    log_diam_unc = data[i+2]
    sma_data = data[i+3]


    
    # Assuming flux is always the first element in the tuple and the logarithmic diameter is the first element in the list
    #if flux_beam[1] != 'NA':  # Skip entries where beam size is 'NA'

    log_diam = float(log_diam_unc[0])  # Use logarithmic diameter
    log_diamunc = float(log_diam_unc[1])
    if galaxy_name not in galaxy_data:
        galaxy_data[galaxy_name] = {'flux': [], 'log_diameter': [], 'fluxunc': [], 'sizeunc': [], 'smaflux': [], 'smafluxunc': []}
    flux=float(flux_beam[0])
    galaxy_data[galaxy_name]['flux'].append(flux)
    galaxy_data[galaxy_name]['log_diameter'].append(log_diam)
    galaxy_data[galaxy_name]['sizeunc'].append(log_diamunc)
    if flux_beam[1] != 'NA':
        fluxunc = float(flux_beam[1])
        galaxy_data[galaxy_name]['fluxunc'].append(fluxunc)
    else:
        galaxy_data[galaxy_name]['fluxunc'].append('NA')

    size = np.size(sma_data[0])
    if not size==1:
        galaxy_data[galaxy_name]['smaflux'].append(sma_data[0][0])
        if sma_data[0][1]!='NA':
            galaxy_data[galaxy_name]['smafluxunc'].append(sma_data[0][1])
        else:
            galaxy_data[galaxy_name]['smafluxunc'].append('NA')
    else:
        galaxy_data[galaxy_name]['smaflux'].append('NA')
        galaxy_data[galaxy_name]['smafluxunc'].append('NA')


# Step 2: Plotting

min_flux = float('inf')  # Initialize min_flux to a very large number
max_flux = float('-inf')  # Initialize max_flux to a very small number
min_log_diam = float('inf')  # Initialize min_log_diam to a very large number
max_log_diam = float('-inf')  # Initialize max_log_diam to a very small number

fig, ax = plt.subplots(figsize=(10, 6), dpi=300)

for galaxy, measurements in galaxy_data.items():

    # Check and update the min/max for flux
    if min(measurements['flux']) < min_flux:
        min_flux = min(measurements['flux'])
    if max(measurements['flux']) > max_flux:
        max_flux = max(measurements['flux'])
    
    # Check and update the min/max for log_diameter
    if min(measurements['log_diameter']) < min_log_diam:
        min_log_diam = min(measurements['log_diameter'])
    if max(measurements['log_diameter']) > max_log_diam:
        max_log_diam = max(measurements['log_diameter'])

    for flux, log_diam, fluxunc, sizeunc, sma, smaunc in zip(measurements['flux'], measurements['log_diameter'], measurements['fluxunc'], measurements['sizeunc'], measurements['smaflux'], measurements['smafluxunc']):
        print(galaxy)
        if fluxunc!='NA':
            plt.errorbar(log_diam, flux, color='blue', markersize=3.5, marker='o',yerr=fluxunc ,xerr=sizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
        else: 
            plt.errorbar(log_diam, flux, color='red', markersize=3.5, marker='v', xerr=sizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
        if sma!='NA':
            if smaunc!='NA'and fluxunc=='NA':
                input('ERROR')
            elif sma==flux and smaunc==fluxunc:
              if smaunc!='NA':
                  pass
                  #plt.errorbar(log_diam, sma, color='white', markersize=1, marker='x')
              else: 
                  pass
                  #plt.errorbar(log_diam, sma, color='white', markersize=1, marker='x')
            else:
                pass
                #print(galaxy)
        if allsourcesofinterest==False:
            for i in sourcesofinterest:
                if i in galaxy:
                    #print(f"{galaxy}: {10**log_diam}")
                    if 'Circinus' in galaxy:
                        plt.text(log_diam, flux-0.12, galaxy, fontsize=9, ha='center')
                    else:
                        plt.text(log_diam, flux+0.03, galaxy, fontsize=9, ha='center')
        else:
            plt.text(log_diam, flux+0.03, galaxy, fontsize=9, ha='center')


if  np.log10(sagsize) > max(measurements['log_diameter']):
    max_log_diam = np.log10(sagsize)

if np.log10(m87size) > max(measurements['log_diameter']):
    max_log_diam = np.log10(m87size)


m87bright=1.571*1000
m87unc=0.261*1000
logm87flux=np.log10(m87bright)
logm87fluxunc=linear_to_log_uncertainty(m87bright, m87unc)

sagbright=3.22*1000
sagunc=0.62*1000
logsagflux=np.log10(sagbright)
logsagfluxunc=linear_to_log_uncertainty(sagbright, sagunc)

logsagsizeunc = np.array([[logsagsizeunc_down], [logsagsizeunc_up]])
logm87sizeunc = np.array([[logm87sizeunc_down], [logm87sizeunc_up]])

if  logm87flux > max_flux:
    max_flux = logm87flux

if logsagflux > max_flux:
    max_flux = logsagflux


min_fluxp=round_down_to_half_integer(min_flux)
max_fluxp=round_up_to_half_integer(max_flux)
min_log_diamp=round_down_to_half_integer(min_log_diam)
max_log_diamp=round_up_to_half_integer(max_log_diam)

interval =  max_fluxp - min_fluxp
ten_percent = interval * 0.05
min_flux = min_fluxp - ten_percent
max_flux = max_fluxp + ten_percent

interval = max_log_diamp - min_log_diamp
ten_percent = interval * 0.05
min_log_diam = min_log_diamp - ten_percent
max_log_diam = max_log_diamp + ten_percent

ax.set_xlim([min_log_diam, max_log_diam])
ax.set_ylim([min_flux, max_flux])

x_tick_positions=np.arange(min_log_diamp, max_log_diamp + 0.5 , 0.5)  # 6 evenly spaced ticks
x_tick_labels = [f"{pos:.1f}" for pos in x_tick_positions]  # Convert log to linear
ax.set_xticks(x_tick_positions)
ax.set_xticklabels(x_tick_labels)

y_tick_positions = np.arange(min_fluxp, max_fluxp + 0.5, 0.5)  # 6 evenly spaced ticks
y_tick_labels = [f"{pos:.1f}" for pos in y_tick_positions]  # Convert log to linear
ax.set_yticks(y_tick_positions)
ax.set_yticklabels(y_tick_labels)

x_minor_locator = MultipleLocator(0.1)
y_minor_locator = MultipleLocator(0.1)
ax.yaxis.set_minor_locator(x_minor_locator)
ax.xaxis.set_minor_locator(y_minor_locator)

for ibase in baselinesize:
    print(f"{ibase[0]}: {ibase[1]}")
    plt.axvline(x=np.log10(ibase[1]), color='black', linestyle='-', linewidth=1)
    plt.text(np.log10(ibase[1])-0.1, max_fluxp, f'{ibase[0]}', verticalalignment='center', color='black', rotation=90)  # Adjust position as needed

for iflux in fluxthresh:
    plt.axhline(y=np.log10(iflux), color='black', linestyle='-', linewidth=1)
    #plt.text(3.1, max(y), 'This is x=3', verticalalignment='center', color='red')  # Adjust position as needed

plt.errorbar(np.log10(sagsize), logsagflux, color='green', markersize=3, marker='o',yerr=logsagfluxunc ,xerr=logsagsizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
plt.text(np.log10(sagsize)+0.31, logsagflux-0.035, f'Sgr A*', ha='center', color='black')  # Adjust position as needed

plt.errorbar(np.log10(m87size), logm87flux, color='green', markersize=3, marker='o',yerr=logm87fluxunc ,xerr=logm87sizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
plt.text(np.log10(m87size)+0.21, logm87flux-0.035, f'M87*', ha='center', color='black')  # Adjust position as needed

#plt.errorbar(np.log10(sizeIC1459), logIC1459flux, color='green', markersize=3, marker='o',yerr=logIC1459fluxunc,xerr=logIC1459sizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
#plt.text(np.log10(sizeIC1459), logIC1459flux+0.07, f'IC 1459', ha='center', color='black', fontsize=9)  # Adjust position as needed

#plt.errorbar(np.log10(sizeNGC4594), logNGC4594flux, color='green', markersize=3, marker='o',yerr=logNGC4594fluxunc,xerr=logNGC4594sizeunc, elinewidth=0.5,capsize=1, capthick=0.5)
#plt.text(np.log10(sizeNGC4594), logNGC4594flux-0.15, f'NGC 4594', ha='center', color='black', fontsize=9)  # Adjust position as needed

for ino in nograph:
    name=ino[0]
    if name=='J0437':
        name='WISEAJ043703\n+245607.0'
    pass
    #plt.axvline(x=ino[1][0], color='black', linestyle='-', linewidth=1, ymin=0, ymax=0.15)
    #plt.text(ino[1][0]+0.05, min_fluxp + 0.15, f'{name}', verticalalignment='center', color='black', rotation=90, fontsize=7)  # Adjust position as needed

plt.xlabel(r'log$_{10}$ Shadow Size ($\mu$")')
plt.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
plt.title(f'BHS vs Flux at 200-400 GHz')
plt.grid(False)
plt.savefig('BHS_Flux_Graph', bbox_inches='tight', dpi=300)
plt.show()